In [4]:
import pandas as pd
import string
import numpy as np
import simpletransformers

In [1]:
pip install simpletransformers

     |████████████████████████████████| 225kB 8.8MB/s 
     |████████████████████████████████| 317kB 11.3MB/s 
     |████████████████████████████████| 7.4MB 13.2MB/s 
     |████████████████████████████████| 1.8MB 37.8MB/s 
     |████████████████████████████████| 2.9MB 38.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 1.4MB 37.8MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 163kB 45.7MB/s 
     |████████████████████████████████| 6.9MB 44.2MB/s 
     |████████████████████████████████| 112kB 40.8MB/s 
     |████████████████████████████████| 4.5MB 33.5MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 133kB 41.7MB/s 
     |████████████████████████████████| 133kB 39.3MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 890kB 35.2MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 


In [3]:
pip install aspect_based_sentiment_analysis

In [8]:
!git clone https://github.com/Navjot862/Courseera_analytics_tools.git

Cloning into 'Courseera_analytics_tools'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 83 (delta 39), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [9]:
Review=pd.read_csv("/content/Courseera_analytics_tools/Train_Data.csv")
Review=Review[['Review']]

In [10]:
##Convert everything to lower cases 
Review['Review']=Review['Review'].str.lower()

In [11]:
Proffessor=['professors','instructor',"ittner","bushee","lecturer","teacher"]

Courses=['autocad','3d printing','traditional tooling','economy','business','course','analytics','accounting','accounting analytics',
         'business analytics', 'business market analysis','advanced neurobiology']

Content=['lecture','videos','content','material','assignments','information']

In [12]:
Review_list=list(Review['Review'])

In [13]:
##Creating an empty data frame
Review_Data_Train=pd.DataFrame(columns=['Sentence_Id','Word','Label'])

In [14]:
for i in range(len(Review_list)):
    Chk=pd.DataFrame(columns=['Sentence_Id','Word','Label'])
#     print(i)
    l2=Review_list[i].split()
    for j in range(len(l2)):
        Chk.at[j, 'Sentence_Id'] = i
        Chk.at[j, 'Word'] = l2[j]
#     print(Review_Data_Train.head(2))
    Review_Data_Train=pd.concat([Review_Data_Train,Chk])
    

In [16]:
Review_Data_Train['Label']='O'

Data=pd.DataFrame(columns=['Sentence_Id','Word','Label'])

for m in np.unique(Review_Data_Train['Sentence_Id']):
#     print(m)
    first_course=0
    first_prof=0
    first_content=0
    Review_Data_Sentence=Review_Data_Train[Review_Data_Train['Sentence_Id']==m]

    for i in range(Review_Data_Sentence.shape[0]):
        
        ##Course
        
        if Review_Data_Sentence['Word'].iloc[i] in Courses and first_course==0:
            Review_Data_Sentence['Label'].iloc[i]='B-Course'
            first_course=first_course+1
            
        elif Review_Data_Sentence['Word'].iloc[i] in Courses and first_course>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-Course'
            first_course=first_course+1   
            
        ##Professor    
        
        if Review_Data_Sentence['Word'].iloc[i] in Proffessor and first_prof==0:
            Review_Data_Sentence['Label'].iloc[i]='B-Prof'
            first_prof=first_prof+1
            
        elif Review_Data_Sentence['Word'].iloc[i] in Proffessor and first_prof>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-Prof'
            first_prof=first_prof+1
        
        ##Content
        
        if Review_Data_Sentence['Word'].iloc[i] in Content and first_content==0:
            Review_Data_Sentence['Label'].iloc[i]='B-Content'
            first_content=first_content+1   
            
            
        elif Review_Data_Sentence['Word'].iloc[i] in Content and first_content>=0:
            Review_Data_Sentence['Label'].iloc[i]='I-Content'
            first_content=first_content+1  
            
    Data=pd.concat([Data,Review_Data_Sentence])        
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to b

In [19]:
np.unique(Data['Sentence_Id'])

array([0, 1, 2, ..., 1747, 1748, 1749], dtype=object)

In [20]:
Train_l1=[]
for x in range(1000):
    Train_l1.append(x)
print(Train_l1)

eval_l1=[]
for x in range(1000,1500):
    eval_l1.append(x)
print(eval_l1)


Test_l1=[]
for x in range(1500,1750):
    Test_l1.append(x)
print(Test_l1)


Train_df=Data[Data['Sentence_Id'].isin(Train_l1)]
eval_df=Data[Data['Sentence_Id'].isin(eval_l1)]
Test_df=Data[Data['Sentence_Id'].isin(Test_l1)]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [21]:
Train_df.columns=["sentence_id", "words", "labels"]
Test_df.columns=["sentence_id", "words", "labels"]
eval_df.columns=["sentence_id", "words", "labels"]

In [22]:
np.unique(Train_df['labels'])

array(['B-Content', 'B-Course', 'B-Prof', 'I-Content', 'I-Course',
       'I-Prof', 'O'], dtype=object)

In [24]:
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
model_args = NERArgs(overwrite_output_dir=True,num_train_epochs=2)
# model_args.train_batch_size = 16
# model_args.evaluate_during_training = True
model_args.labels_list=list(np.unique(Train_df['labels']))

model = NERModel("roberta", "roberta-base", args=model_args)

# Train the model

model.train_model(Train_df, eval_data=eval_df)
# model.train_model(Train_df,eval_df)

# # Evaluate the model
# result, model_outputs, preds_list = model.eval_model(eval_df)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning:

Please also save or load the state of the optimizer when saving or loading the scheduler.



(250, 0.13674915226269513)

In [27]:
from sklearn.metrics import accuracy_score

In [ ]:
Sample_Review=pd.read_csv("/content/Courseera_analytics_tools/Sample_Review.csv")

In [ ]:
entity_df=pd.DataFrame(columns=['Text','Entity','Sentiment','Score(Neutral/Negative/Positive)'])

for  i in range(Sample_Review.shape[0]):
  Text1=Sample_Review['Review'].iloc[i]
  Text1=Text1.lower()
  a=[]
  a.append(Text1)
  pred1,raw_op_1 = model.predict(a)
  # print(pred1)
  entity=[]
  for j in range(len(pred1[0])):
    if list(pred1[0][j].values())[0]!='O' and list(pred1[0][j].keys())[0] not in entity:
      # print(list(pred1[0][j].keys())[0])
      # print (pred1)
      entity.append(list(pred1[0][j].keys())[0])
  entity_df.at[i,'Text']=Text1 
  entity_df.at[i,'Entity']=entity    
  # print(entity) 


In [37]:
Sample_Review=pd.read_csv("/content/Courseera_analytics_tools/Sample_Review.csv")

In [39]:
import aspect_based_sentiment_analysis as absa
nlp = absa.load()

ValueError: ignored

In [38]:
##club loops

##Making an entity dataframe

entity_df=pd.DataFrame(columns=['Text','Entity','Sentiment','Score(Neutral/Negative/Positive)'])

##loop for Review
for  i in range(Sample_Review.shape[0]):
  ##Creating Text column for filtering
  Text1=Sample_Review['Review'].iloc[i]
  Text1=Text1.lower()
  a=[]
  a.append(Text1)
  ##Entity Prediction
  pred1,raw_op_1 = model.predict(a)
  # print(pred1)
  entity=[]
  ##Loop for predicttion
  for j in range(len(pred1[0])):
    if list(pred1[0][j].values())[0]!='O' and list(pred1[0][j].keys())[0] not in entity:
      # print(list(pred1[0][j].keys())[0])
      # print (pred1)
      entity.append(list(pred1[0][j].keys())[0])
  # print(entity)    
  ##Creating list of sentiment and score for sentiment   
  Sentiment_list=[]
  Score_list=[]
  ##loop for travessing entity 
  for k in range(len(entity)):
    en=nlp(Text1,aspects=[entity[k]])
    b=list(en.subtasks.values())[0]  
    Sentiment=str(b.aspect)+"="+str(b.sentiment)
    Scores=str(b.aspect)+"="+str(b.scores)
    Sentiment_list.append(Sentiment)
    Score_list.append(Scores)
  ##Storing values in data frame   
  entity_df.at[i,'Text']=Text1 
  entity_df.at[i,'Entity']=entity 
  entity_df.at[i,'Sentiment']=Sentiment_list 
  entity_df.at[i,'Score(Neutral/Negative/Positive)']=Score_list    


NameError: ignored

In [ ]:
entity_df

,Text,Entity,Sentiment,Score(Neutral/Negative/Positive)
0,is there any reason why you should not apply t...,"[course, content]","[course=Sentiment.neutral, content=Sentiment.p...","[course=[0.5876564, 0.051309723, 0.36103392], ..."
1,"excellent course, for me it was very rewarding...",[],[],[]
2,great course information and content ! very in...,"[course, information]","[course=Sentiment.positive, information=Sentim...","[course=[0.0023027516, 0.0016259098, 0.9960713..."
3,this has been an phenomenal course on the educ...,[course],[course=Sentiment.positive],"[course=[0.0005151652, 0.00094915077, 0.9985357]]"
4,well-presented contents with comprehensive ill...,[],[],[]
5,"progress is in a state of flux, so one must re...",[course],[course=Sentiment.positive],"[course=[0.0012729622, 0.00096010364, 0.997767]]"
6,a really challenging and enriching course abou...,[course],[course=Sentiment.positive],"[course=[0.0006520916, 0.001040037, 0.9983079]]"
7,this is a really good survey class in the pote...,[],[],[]
8,topics related to the economy or business are ...,"[business, course]","[business=Sentiment.neutral, course=Sentiment....","[business=[0.57312536, 0.37201113, 0.05486352]..."
9,"this course is very interesting, enlightening ...",[course],[course=Sentiment.positive],"[course=[0.00063709315, 0.0009874566, 0.9983755]]"


In [ ]:
entity_df.to_csv("/content/Courseera_analytics_tools/entity.csv")

In [ ]:
Text2="I didn't like that course because of:-weak explanations of algorithms-almost all assignments I googled, I spent all my time googling these algorithms-when code is submitted and something goes wrong It is not possible to understand what is really happened"
Text2=Text2.lower()
a=[]
a.append(Text2)
pred2,raw_op_2 = model.predict(a)
print(pred2)

course,assignments = nlp(Text2, aspects=['course', 'assignments'])
print(absa.summary(course))
print(absa.summary(assignments))


In [ ]:
Text1="Professors need to undergo a presentation and Instruction Design skills. They were talking a lot and everything was going over my head, the reason was there were not visual prompt for the examples, specifically in the last week."
Text1=Text1.lower()
a=[]
a.append(Text1)
pred1,raw_op_1 = model.predict(a)
print(pred1)

prof,instruction = nlp(Text1, aspects=['professors','instruction'])
print(absa.summary(prof))
print(absa.summary(instruction))


[[{'professors': 'B-Prof'}, {'need': 'O'}, {'to': 'O'}, {'undergo': 'O'}, {'a': 'O'}, {'presentation': 'O'}, {'and': 'O'}, {'instruction': 'O'}, {'design': 'O'}, {'skills.': 'O'}, {'they': 'O'}, {'were': 'O'}, {'talking': 'O'}, {'a': 'O'}, {'lot': 'O'}, {'and': 'O'}, {'everything': 'O'}, {'was': 'O'}, {'going': 'O'}, {'over': 'O'}, {'my': 'O'}, {'head,': 'O'}, {'the': 'O'}, {'reason': 'O'}, {'was': 'O'}, {'there': 'O'}, {'were': 'O'}, {'not': 'O'}, {'visual': 'O'}, {'prompt': 'O'}, {'for': 'O'}, {'the': 'O'}, {'examples,': 'O'}, {'specifically': 'O'}, {'in': 'O'}, {'the': 'O'}, {'last': 'O'}, {'week.': 'O'}]]
Sentiment.negative for "professors"
Scores (neutral/negative/positive): [0.034 0.958 0.008]
None
Sentiment.negative for "instruction"
Scores (neutral/negative/positive): [0.255 0.72  0.026]
None


In [ ]:
prof = nlp(Text1, aspects=['professors'])

In [ ]:
a=list(prof.subtasks.values())[0]

In [ ]:
a.aspect

'professors'

In [ ]:
a.scores

[0.0338635, 0.958304, 0.007832578]

In [ ]:
a.sentiment

<Sentiment.negative: 1>

In [ ]:
a.text

"i've already written a review for part 1 and i have the same opinion about this one. the course is rather poor and not challenging. only general information about relevant topics that as well read on wikipedia. no exercises, no code assignments. a lot of this content was repeated from first two parts of this specialization."

In [ ]:
str(a.aspect)+"  " +str(a.sentiment)+" "+"neutral/negative/positive "+str(a.scores)

'professors  Sentiment.negative neutral/negative/positive [0.0338635, 0.958304, 0.007832578]'

In [ ]:
absa.summary(a)

Sentiment.negative for "professors"
Scores (neutral/negative/positive): [0.034 0.958 0.008]


In [ ]:
list(prof.subtasks.values())[0].scores

[0.0338635, 0.958304, 0.007832578]

In [ ]:
list(prof.subtasks.values())[0]

[0.0338635, 0.958304, 0.007832578]

In [ ]:
list(a)[0].sentiment

<Sentiment.negative: 1>

In [ ]:
a.sentiment

<Sentiment.negative: 1>

In [ ]:
for key, value in d1.items(): 
  a=value
  

In [ ]:
type(d1)

collections.OrderedDict

In [ ]:
absa.summary(prof)

Sentiment.negative for "professors"
Scores (neutral/negative/positive): [0.034 0.958 0.008]


In [ ]:
?CompletedSubTask

Object `CompletedSubTask` not found.


In [ ]:
absa.summary(aa)

Sentiment.negative for "aaaa"
Scores (neutral/negative/positive): [0.006 0.99  0.005]


In [ ]:
Text2="I didn't like that course because of:-weak explanations of algorithms-almost all assignments I googled, I spent all my time googling these algorithms-when code is submitted and something goes wrong It is not possible to understand what is really happened"
Text2=Text2.lower()
a=[]
a.append(Text2)
pred2,raw_op_2 = model.predict(a)
print(pred2)

course,assignments = nlp(Text2, aspects=['course', 'assignments'])
print(absa.summary(course))
print(absa.summary(assignments))

# absa.Sentiment(course)
# print(absa.summary(course))


[[{'i': 'O'}, {"didn't": 'O'}, {'like': 'O'}, {'that': 'O'}, {'course': 'B-Course'}, {'because': 'O'}, {'of:-weak': 'O'}, {'explanations': 'O'}, {'of': 'O'}, {'algorithms-almost': 'O'}, {'all': 'O'}, {'assignments': 'B-Content'}, {'i': 'O'}, {'googled,': 'O'}, {'i': 'O'}, {'spent': 'O'}, {'all': 'O'}, {'my': 'O'}, {'time': 'O'}, {'googling': 'O'}, {'these': 'O'}, {'algorithms-when': 'O'}, {'code': 'O'}, {'is': 'O'}, {'submitted': 'O'}, {'and': 'O'}, {'something': 'O'}, {'goes': 'O'}, {'wrong': 'O'}, {'it': 'O'}, {'is': 'O'}, {'not': 'O'}, {'possible': 'O'}, {'to': 'O'}, {'understand': 'O'}, {'what': 'O'}, {'is': 'O'}, {'really': 'O'}, {'happened': 'O'}]]
Sentiment.negative for "course"
Scores (neutral/negative/positive): [0.004 0.984 0.011]
None
Sentiment.negative for "assignments"
Scores (neutral/negative/positive): [0.053 0.938 0.009]
None


In [ ]:
Text3="I was disappointed because the name is misleading. The course provides a good introduction & overview of the responsibilities of the CTO, but has very little specifically digital content. It deals with two-speed IT in a single short lecture, so of course the treatment is superficial. It is easy to find more in-depth material freely available, on the McKinsey website for example."
Text3=Text3.lower()
a=[]
a.append(Text3)
pred3,raw_op_2 = model.predict(a)
print(pred3)

course,material = nlp(Text3, aspects=['course', 'material'])
print(absa.summary(course))
print(absa.summary(material))


[[{'i': 'O'}, {'was': 'O'}, {'disappointed': 'O'}, {'because': 'O'}, {'the': 'O'}, {'name': 'O'}, {'is': 'O'}, {'misleading.': 'O'}, {'the': 'O'}, {'course': 'B-Course'}, {'provides': 'O'}, {'a': 'O'}, {'good': 'O'}, {'introduction': 'O'}, {'&': 'O'}, {'overview': 'O'}, {'of': 'O'}, {'the': 'O'}, {'responsibilities': 'O'}, {'of': 'O'}, {'the': 'O'}, {'cto,': 'O'}, {'but': 'O'}, {'has': 'O'}, {'very': 'O'}, {'little': 'O'}, {'specifically': 'O'}, {'digital': 'O'}, {'content.': 'O'}, {'it': 'O'}, {'deals': 'O'}, {'with': 'O'}, {'two-speed': 'O'}, {'it': 'O'}, {'in': 'O'}, {'a': 'O'}, {'single': 'O'}, {'short': 'O'}, {'lecture,': 'O'}, {'so': 'O'}, {'of': 'O'}, {'course': 'I-Course'}, {'the': 'O'}, {'treatment': 'O'}, {'is': 'O'}, {'superficial.': 'O'}, {'it': 'O'}, {'is': 'O'}, {'easy': 'O'}, {'to': 'O'}, {'find': 'O'}, {'more': 'O'}, {'in-depth': 'O'}, {'material': 'I-Content'}, {'freely': 'O'}, {'available,': 'O'}, {'on': 'O'}, {'the': 'O'}, {'mckinsey': 'O'}, {'website': 'O'}, {'for'

In [ ]:
Text4="I love the professors, but unfortunately, I do not feel much connection to me (not really seeing so much importance for me to identifying if a company is making fake financial statements; I am not an investor, and my everyday work does not really have so much to do with financial statements).The Managerial Accounting (week 4) transforms my mindset!! I really appreciate that. That one single week of course can literally help so many companies/NGOs. That week 4 should be promoted more!!"
Text4=Text4.lower()
a=[]
a.append(Text4)
pred4,raw_op_4 = model.predict(a)
print(pred4)

professors,course,accounting = nlp(Text4, aspects=['professors', 'course','accounting'])
print(absa.summary(professors))
print(absa.summary(course))
print(absa.summary(accounting))


[[{'i': 'O'}, {'love': 'O'}, {'the': 'O'}, {'professors,': 'O'}, {'but': 'O'}, {'unfortunately,': 'O'}, {'i': 'O'}, {'do': 'O'}, {'not': 'O'}, {'feel': 'O'}, {'much': 'O'}, {'connection': 'O'}, {'to': 'O'}, {'me': 'O'}, {'(not': 'O'}, {'really': 'O'}, {'seeing': 'O'}, {'so': 'O'}, {'much': 'O'}, {'importance': 'O'}, {'for': 'O'}, {'me': 'O'}, {'to': 'O'}, {'identifying': 'O'}, {'if': 'O'}, {'a': 'O'}, {'company': 'O'}, {'is': 'O'}, {'making': 'O'}, {'fake': 'O'}, {'financial': 'O'}, {'statements;': 'O'}, {'i': 'O'}, {'am': 'O'}, {'not': 'O'}, {'an': 'O'}, {'investor,': 'O'}, {'and': 'O'}, {'my': 'O'}, {'everyday': 'O'}, {'work': 'O'}, {'does': 'O'}, {'not': 'O'}, {'really': 'O'}, {'have': 'O'}, {'so': 'O'}, {'much': 'O'}, {'to': 'O'}, {'do': 'O'}, {'with': 'O'}, {'financial': 'O'}, {'statements).the': 'O'}, {'managerial': 'O'}, {'accounting': 'O'}, {'(week': 'O'}, {'4)': 'O'}, {'transforms': 'O'}, {'my': 'O'}, {'mindset!!': 'O'}, {'i': 'O'}, {'really': 'O'}, {'appreciate': 'O'}, {'tha

In [ ]:
pred4

[[{'i': 'O'},
  {'love': 'O'},
  {'the': 'O'},
  {'professors,': 'O'},
  {'but': 'O'},
  {'unfortunately,': 'O'},
  {'i': 'O'},
  {'do': 'O'},
  {'not': 'O'},
  {'feel': 'O'},
  {'much': 'O'},
  {'connection': 'O'},
  {'to': 'O'},
  {'me': 'O'},
  {'(not': 'O'},
  {'really': 'O'},
  {'seeing': 'O'},
  {'so': 'O'},
  {'much': 'O'},
  {'importance': 'O'},
  {'for': 'O'},
  {'me': 'O'},
  {'to': 'O'},
  {'identifying': 'O'},
  {'if': 'O'},
  {'a': 'O'},
  {'company': 'O'},
  {'is': 'O'},
  {'making': 'O'},
  {'fake': 'O'},
  {'financial': 'O'},
  {'statements;': 'O'},
  {'i': 'O'},
  {'am': 'O'},
  {'not': 'O'},
  {'an': 'O'},
  {'investor,': 'O'},
  {'and': 'O'},
  {'my': 'O'},
  {'everyday': 'O'},
  {'work': 'O'},
  {'does': 'O'},
  {'not': 'O'},
  {'really': 'O'},
  {'have': 'O'},
  {'so': 'O'},
  {'much': 'O'},
  {'to': 'O'},
  {'do': 'O'},
  {'with': 'O'},
  {'financial': 'O'},
  {'statements).the': 'O'},
  {'managerial': 'O'},
  {'accounting': 'O'},
  {'(week': 'O'},
  {'4)': 'O'},


In [ ]:
Text5="First of all, I really enjoyed Professor Ittner presentations. Sadly, his presentation was only 25% of the course. Professor Ittner was very direct but also was very engaging in how he delivered his content."
#  I had a good feeling that I could really understand the material he was teaching. If I could give a rating for just Professor Ittner, I would give a 5 star rating. Now, this is why I am giving this course a 1 star rating. But I do have to say, going into the depths of Accounting Analytics is not easy. You really need to have a good grasp of Accounting otherwise you will get lost. Professor Bushee did not make the first 3 weeks really easy to get through. While I enjoyed his examples & case studies, he simply did not do a great job of really explaining and illustrating the concepts he was teaching. And worst of all, he setup a digitally generated classroom where he had his computer generated students ask pointless questions, make pointless statements, or answer with pointless statements/answers. I really wish I had the time to analyze how much of the course time he spent doing this, I would estimate that its around 10% but it could very well be higher. There are so many different examples to pick form, but I ll just pick a few here. I didn't read the Iliad or Odyssey some I'm not going to ask you to in reference why in regression there are Greek letter used for coefficients (its a fair assumption that someone taking this class understands coefficients), a 30 second exchange where a PC generate student says that they're "stocked" to run some regression followed by Proffessor Bushee informing us that "we're going to run some narly regressions, Professor Bushee saying he doesn t know how to perform Earning Management Analysis on Mars but he certainly knows how to do it on Earth, and a joke why people dont like accountants. These segments add absolutely zero value. They lengthen the time of the course. To make maters worse, I m sure Proffessor Bushee had to spend even more time creating this additional interaction together."
Text5=Text5.lower()
a=[]
a.append(Text5)
pred5,raw_op_5 = model.predict(a)
print(pred5)

# professors,course,accounting = nlp(Text4, aspects=['professors', 'course','accounting'])
# print(absa.summary(professors))
# print(absa.summary(course))
# print(absa.summary(accounting))


[[{'first': 'O'}, {'of': 'O'}, {'all,': 'O'}, {'i': 'O'}, {'really': 'O'}, {'enjoyed': 'O'}, {'professor': 'O'}, {'ittner': 'O'}, {'presentations.': 'O'}, {'sadly,': 'O'}, {'his': 'O'}, {'presentation': 'O'}, {'was': 'O'}, {'only': 'O'}, {'25%': 'O'}, {'of': 'O'}, {'the': 'O'}, {'course.': 'O'}, {'professor': 'O'}, {'ittner': 'O'}, {'was': 'O'}, {'very': 'O'}, {'direct': 'O'}, {'but': 'O'}, {'also': 'O'}, {'was': 'O'}, {'very': 'O'}, {'engaging': 'O'}, {'in': 'O'}, {'how': 'O'}, {'he': 'O'}, {'delivered': 'O'}, {'his': 'O'}, {'content.': 'O'}]]


In [ ]:
pred5

[[{'first': 'O'},
  {'of': 'O'},
  {'all,': 'O'},
  {'i': 'O'},
  {'really': 'O'},
  {'enjoyed': 'O'},
  {'professor': 'O'},
  {'ittner': 'O'},
  {'presentations.': 'O'},
  {'sadly,': 'O'},
  {'his': 'O'},
  {'presentation': 'O'},
  {'was': 'O'},
  {'only': 'O'},
  {'25%': 'O'},
  {'of': 'O'},
  {'the': 'O'},
  {'course.': 'O'},
  {'professor': 'O'},
  {'ittner': 'O'},
  {'was': 'O'},
  {'very': 'O'},
  {'direct': 'O'},
  {'but': 'O'},
  {'also': 'O'},
  {'was': 'O'},
  {'very': 'O'},
  {'engaging': 'O'},
  {'in': 'O'},
  {'how': 'O'},
  {'he': 'O'},
  {'delivered': 'O'},
  {'his': 'O'},
  {'content.': 'O'}]]

In [ ]:
import aspect_based_sentiment_analysis as absa

nlp = absa.load()
text = ("The course is fine as far as content. My complaint is that there is no one to contact regarding papers or questions. You are left to the other students for your grading and assignment completion, meaning that some of your assignments can hang there for weeks before ever getting reviewed.")

course,assignments = nlp(text, aspects=['course', 'assignments'])
print(absa.summary(course))
print(absa.summary(assignments))

Some layers from the model checkpoint at absa/classifier-rest-0.2 were not used when initializing BertABSClassifier: ['dropout_379']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertABSClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of BertABSClassifier were not initialized from the model checkpoint at absa/classifier-rest-0.2 and are newly initialized: ['dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Sentiment.positive for "course"
Scores (neutral/negative/positive): [0.003 0.001 0.996]
None
Sentiment.neutral for "assignments"
Scores (neutral/negative/positive): [0.59  0.401 0.008]
None


In [ ]:
absa.summary(course)

Sentiment.positive for "course"
Scores (neutral/negative/positive): [0.003 0.001 0.996]


In [ ]:
absa.summary(assignments)

Sentiment.neutral for "assignments"
Scores (neutral/negative/positive): [0.59  0.401 0.008]


In [ ]:
!pip install aspect-based-sentiment-analysis

     |████████████████████████████████| 266kB 9.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92kB 7.7MB/s 
     |████████████████████████████████| 163kB 20.6MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 2.9MB 19.6MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 133kB 44.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 112kB 42.1MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=50c55399bf4f19d1e308023d2e5becaa2e8b38ecb5ce9223584adbcbae621750
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13700 sha25